## 9.2.1. Scopes and Namespaces Example

In [2]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


## 9.3.1. Class Definition Syntax

Class definitions, like function definitions (def statements) must be executed before they have any effect. (You could conceivably place a class definition in a branch of an if statement, or inside a function.)

## 9.3.2. Class Objects

In [3]:
class MyClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'

MyClass.i and MyClass.f are valid attribute references, returning an integer and a function object, respectively. \_\_doc\_\_ is also a valid attribute, returning the docstring belonging to the class: "A simple example class".

Class instantiation uses function notation. Just pretend that the class object is a parameterless function that returns a new instance of the class. For example (assuming the above class):

In [4]:
x = MyClass()

Many classes like to create objects with instances customized to a specific initial state. Therefore a class may define a special method named __init__(), like this:

In [6]:
class Complex:
    def __init__(self, realpart, imagpart):
        self.r = realpart
        self.i = imagpart

x = Complex(3.0, -4.5)
x.r, x.i

(3.0, -4.5)

In [7]:
Complex()

TypeError: __init__() missing 2 required positional arguments: 'realpart' and 'imagpart'

## 9.3.3. Instance Objects

In [12]:
x.counter = 1
while x.counter < 10:
    x.counter = x.counter * 2
print(x.counter)
del x.counter

16


## 9.3.4. Method Objects

In [16]:
class MyClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'
    
x = MyClass()
xf = x.f
print(xf())
MyClass.f(x)

hello world


'hello world'

## 9.3.5. Class and Instance Variables

In [20]:
class Dog:

    kind = 'canine'         # class variable shared by all instances

    def __init__(self, name):
        self.name = name    # instance variable unique to each instance

d = Dog('Fido')
e = Dog('Buddy')
print(d.kind)                  # shared by all dogs
print(e.kind)                  # shared by all dogs
print(d.name)                  # unique to d
print(e.name)                  # unique to e

canine
canine
Fido
Buddy


In [21]:
class Dog:
    tricks = []             # mistaken use of a class variable

    def __init__(self, name):
        self.name = name

    def add_trick(self, trick):
        self.tricks.append(trick)
        
d = Dog('Fido')
e = Dog('Buddy')
d.add_trick('roll over')
e.add_trick('play dead')
d.tricks                # unexpectedly shared by all dogs

['roll over', 'play dead']

Correct design of the class should use an instance variable instead:

In [23]:
class Dog:
    def __init__(self, name):
        self.name = name
        self.tricks = []

    def add_trick(self, trick):
        self.tricks.append(trick)
        
d = Dog('Fido')
e = Dog('Buddy')
d.add_trick('roll over')
e.add_trick('play dead')
print(d.tricks)
print(e.tricks)

['roll over']
['play dead']


## 9.4. Random Remarks

Data attributes override method attributes with the same name; to avoid accidental name conflicts, which may cause hard-to-find bugs in large programs, it is wise to use some kind of convention that minimizes the chance of conflicts. Possible conventions include capitalizing method names, prefixing data attribute names with a small unique string (perhaps just an underscore), or using verbs for methods and nouns for data attributes.

Nothing in Python makes it possible to enforce data hiding — it is all based upon convention. 

In [25]:
# Function defined outside the class
def f1(self, x, y):
    return min(x, x+y)

class C:
    f = f1

    def g(self):
        return 'hello world'

    h = g

Now f, g and h are all attributes of class C that refer to function objects, and consequently they are all methods of instances of C — h being exactly equivalent to g. Note that this practice usually only serves to confuse the reader of a program.

In [27]:
class Bag:
    def __init__(self):
        self.data = []

    def add(self, x):
        self.data.append(x)

    def addtwice(self, x):
        self.add(x)
        self.add(x)

In [28]:
bag = Bag()
print(bag.__class__)

<class '__main__.Bag'>


## 9.5. Inheritance

In [29]:
class DerivedClassName(modname.BaseClassName):

SyntaxError: unexpected EOF while parsing (<ipython-input-29-f22528feb0f2>, line 1)

There is a simple way to call the base class method directly: just call BaseClassName.methodname(self, arguments). This is occasionally useful to clients as well. (Note that this only works if the base class is accessible as BaseClassName in the global scope.)

Python has two built-in functions that work with inheritance:
- Use isinstance() to check an instance’s type: isinstance(obj, int) will be True only if obj.\_\_class\_\_ is int or some class derived from int.
- Use issubclass() to check class inheritance: issubclass(bool, int) is True since bool is a subclass of int. However, issubclass(float, int) is False since float is not a subclass of int.

## 9.5.1. Multiple Inheritance

In [30]:
class DerivedClassName(Base1, Base2, Base3):
    <statement-1>
    .
    .
    .
    <statement-N>

SyntaxError: invalid syntax (<ipython-input-30-267b91c8270e>, line 2)

For most purposes, in the simplest cases, you can think of the search for attributes inherited from a parent class as depth-first, left-to-right, not searching twice in the same class where there is an overlap in the hierarchy. Thus, if an attribute is not found in DerivedClassName, it is searched for in Base1, then (recursively) in the base classes of Base1, and if it was not found there, it was searched for in Base2, and so on.

## 9.6. Private Variables

Since there is a valid use-case for class-private members (namely to avoid name clashes of names with names defined by subclasses), there is limited support for such a mechanism, called name mangling. Any identifier of the form \_\_spam (at least two leading underscores, at most one trailing underscore) is textually replaced with \_classname__spam, where classname is the current class name with leading underscore(s) stripped. This mangling is done without regard to the syntactic position of the identifier, as long as it occurs within the definition of a class.

In [31]:
class Mapping:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)

    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)

    __update = update   # private copy of original update() method

class MappingSubclass(Mapping):

    def update(self, keys, values):
        # provides new signature for update()
        # but does not break __init__()
        for item in zip(keys, values):
            self.items_list.append(item)

Notice that code passed to exec() or eval() does not consider the classname of the invoking class to be the current class; this is similar to the effect of the global statement, the effect of which is likewise restricted to code that is byte-compiled together. The same restriction applies to getattr(), setattr() and delattr(), as well as when referencing \_\_dict__ directly.

## 9.7. Odds and Ends

In [32]:
class Employee:
    pass

john = Employee()  # Create an empty employee record

# Fill the fields of the record
john.name = 'John Doe'
john.dept = 'computer lab'
john.salary = 1000

A piece of Python code that expects a particular abstract data type can often be passed a class that emulates the methods of that data type instead. For instance, if you have a function that formats some data from a file object, you can define a class with methods read() and readline() that get the data from a string buffer instead, and pass it as an argument.

## 9.8. Iterators

In [33]:
for element in [1, 2, 3]:
    print(element)
for element in (1, 2, 3):
    print(element)
for key in {'one':1, 'two':2}:
    print(key)
for char in "123":
    print(char)
for line in open("myfile.txt"):
    print(line, end='')

1
2
3
1
2
3
one
two
1
2
3


FileNotFoundError: [Errno 2] No such file or directory: 'myfile.txt'

This style of access is clear, concise, and convenient. The use of iterators pervades and unifies Python. Behind the scenes, the for statement calls iter() on the container object. The function returns an iterator object that defines the method \_\_next__() which accesses elements in the container one at a time. When there are no more elements, \_\_next__() raises a StopIteration exception which tells the for loop to terminate. You can call the \_\_next__() method using the next() built-in function; this example shows how it all works:

In [35]:
s = 'abc'
it = iter(s)
it

In [36]:
next(it)

'a'

In [37]:
next(it)

'b'

In [38]:
next(it)

'c'

In [39]:
next(it)

StopIteration: 

In [40]:
class Reverse:
    """Iterator for looping over a sequence backwards."""
    def __init__(self, data):
        self.data = data
        self.index = len(data)

    def __iter__(self):
        return self

    def __next__(self):
        if self.index == 0:
            raise StopIteration
        self.index = self.index - 1
        return self.data[self.index]

In [41]:
rev = Reverse('spam')
iter(rev)

In [42]:
for char in rev:
    print(char)

m
a
p
s


## 9.9. Generators

In [43]:
def reverse(data):
    for index in range(len(data)-1, -1, -1):
        yield data[index]

for char in reverse('golf'):
    print(char)

f
l
o
g


In [46]:
list(range(len("golf")-1, -1, -1))

[3, 2, 1, 0]

What makes generators so compact is that the \_\_iter__() and \_\_next__() methods are created automatically. Another key feature is that the local variables and execution state are automatically saved between calls. This made the function easier to write and much more clear than an approach using instance variables like self.index and self.data.

In addition to automatic method creation and saving program state, when generators terminate, they automatically raise StopIteration. In combination, these features make it easy to create iterators with no more effort than writing a regular function.

## 9.10. Generator Expressions

Some simple generators can be coded succinctly as expressions using a syntax similar to list comprehensions but with parentheses instead of square brackets. These expressions are designed for situations where the generator is used right away by an enclosing function. Generator expressions are more compact but less versatile than full generator definitions and tend to be more memory friendly than equivalent list comprehensions.

In [47]:
sum(i*i for i in range(10))                 # sum of squares

285

In [48]:
xvec = [10, 20, 30]
yvec = [7, 5, 3]
sum(x*y for x,y in zip(xvec, yvec))         # dot product

260

In [49]:
from math import pi, sin
sine_table = {x: sin(x*pi/180) for x in range(0, 91)}

In [54]:
unique_words = set("word  for line in page  for word in line".split())

In [56]:
valedictorian = max((student.gpa, student.name) for student in graduates)

NameError: name 'graduates' is not defined

In [51]:
data = 'golf'
list(data[i] for i in range(len(data)-1, -1, -1))

['f', 'l', 'o', 'g']

In [52]:
[i*i for i in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [53]:
sine_table

{0: 0.0,
 1: 0.01745240643728351,
 2: 0.03489949670250097,
 3: 0.05233595624294383,
 4: 0.0697564737441253,
 5: 0.08715574274765817,
 6: 0.10452846326765346,
 7: 0.12186934340514748,
 8: 0.13917310096006544,
 9: 0.15643446504023087,
 10: 0.17364817766693033,
 11: 0.1908089953765448,
 12: 0.20791169081775931,
 13: 0.224951054343865,
 14: 0.24192189559966773,
 15: 0.25881904510252074,
 16: 0.27563735581699916,
 17: 0.29237170472273677,
 18: 0.3090169943749474,
 19: 0.32556815445715664,
 20: 0.3420201433256687,
 21: 0.35836794954530027,
 22: 0.374606593415912,
 23: 0.3907311284892737,
 24: 0.40673664307580015,
 25: 0.42261826174069944,
 26: 0.4383711467890774,
 27: 0.45399049973954675,
 28: 0.4694715627858908,
 29: 0.48480962024633706,
 30: 0.49999999999999994,
 31: 0.5150380749100542,
 32: 0.5299192642332049,
 33: 0.5446390350150271,
 34: 0.5591929034707469,
 35: 0.573576436351046,
 36: 0.5877852522924731,
 37: 0.6018150231520483,
 38: 0.6156614753256582,
 39: 0.6293203910498374,
 40: 0.

In [55]:
unique_words

{'for', 'in', 'line', 'page', 'word'}